# Experiment Notebook - New Test
This notebook will illustrate how to create and run a new experiment, or Test Run.

In [1]:
import sqlite3
# Create a connection to the database
db_connection = sqlite3.connect('../experiment.db')

# set info level logging
from logging import basicConfig, INFO
basicConfig(level=INFO)


## A. Creating a new Test Run
Each experiment is represented by a Test Run, which will consume a Dataset to perform completions on a related set of Questions.

The generated Responses and their Contexts will then be evaluated by the configured Test Eval functions.

In [2]:
# Create a new Test Run
from eval_data.models.testrun import TestRunModel, TestRunType
from eval_data.models.datasource import DatasourceModel

datasource = DatasourceModel(db_connection).get_datasource_by_name("WikiQA")
print(f"Datasource ID: {datasource.id}")

test_run = TestRunModel(db_connection).add_or_get_test_run(
    TestRunType(
        datasource_id=datasource.id,
        description="Simple RAG test on a small data set"
    )
)

print(f"Test Run ID: {test_run.id}")


Datasource ID: 3
Test Run ID: 2


## B. Load document text index


In [3]:
from typing import List
from eval_data.models.question import QuestionModel, QuestionType
from eval_data.models.document import DocumentModel


documents = DocumentModel(db_connection).get_documents_by_datasource(datasource_id=test_run.datasource_id)

questions: List[QuestionType] = []
for doc in documents:
    questions.extend(QuestionModel(db_connection).get_questions_by_document_id(document_id=doc.id))

print(f"Loaded {len(documents)} documents")

Loaded 1 documents


## TODO: C: Save text and embeddings to db

In [4]:
from eval_scripts.documents import load_documents

nodes = load_documents(db_connection, documents)

print(f"Loaded {len(nodes)} nodes")

/Users/space/Documents/projects/dr-eval/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/space/Documents/projects/dr-eval/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


KeyboardInterrupt: 

## D: Create instance of the LLM query engine

In [ ]:
from my_generator import build_query_engine

query_engine = build_query_engine(nodes)

## E: Response Generation
Utilizing the RAG system to generate responses to the questions during test runs.

In [ ]:
from packages.data.src.eval_data.models.response import ResponseModel, ResponseType
from packages.data.src.eval_data.models.context import ContextModel, ContextType

# Generate responses
for q in questions:
    res = query_engine.query(q.question)
    response = ResponseType(
        test_run_id = test_run.id,
        question_id = q.id,
        response = res.response
    )
    response_id = ResponseModel(db_connection).add_response(response)
    print(f"Response created: {response_id} - len: {len(response.response)}")

    context_model = ContextModel(db_connection)
    for c in res.source_nodes:
        context_model.add_context(
            ContextType(
                response_id=response_id, 
                text=c.node.get_content(),
                similarity_score=c.score
            )
        )